# Sci Kit-Learn Work Flow: Grid Search and Cross Validation 
Grid Search is a powerful tool to compare models with differing parameters. 
This module will be completed shortly.

In [ ]:
http://scikit-learn.org/stable/modules/grid_search.html
http://scikit-learn.org/stable/modules/cross_validation.html
